# merging DSS_SINIESTROS_AUTOS + (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# general configs
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Merging DSS_SINIESTROS_AUTOS + merged (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

## Loading DSS_SINIESTROS_AUTOS

In [11]:
# path to dataset
path_sini = "../../../data/interim/4. transformed/DSS_SINI_AUTOS_ID-red_col-newcols-dropDuplic-NUM_SECU_EXPED_UNIC.feather"

if not(os.path.exists(path_sini) and os.path.isfile(path_sini)):
    raise Exception("File {} doesn't exists.".format(path_sini))

In [12]:
df_sini = pd.read_feather(path_sini)
df_sini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144995 entries, 0 to 1144994
Data columns (total 44 columns):
CATASTROFICO              1144989 non-null object
CERRADURA_BAUL            504402 non-null object
CERRADURA_DERECHA         504402 non-null object
CERRADURA_IZQUIERDA       504402 non-null object
CODIGO_BAJA               1562 non-null float64
CODIGO_CARATULA           30240 non-null float64
CODIGO_REAPERTURA         58512 non-null float64
CODIGO_VEHICULO           1144905 non-null float64
COD_ACT_BENEF             463893 non-null float64
COD_CAUSA_SINI            1144995 non-null float64
COD_POST_OCURRENCIA       1144991 non-null float64
COD_POST_POLIZA           1144971 non-null float64
COD_POST_TERC             194056 non-null float64
COD_RAMO                  1144995 non-null int64
COD_RIES                  1144995 non-null float64
CONDICION_ROBO_EXP50      13085 non-null float64
DANOS_MATERIALES          1144989 non-null object
DESCRIPCION_TIPO          1144993 non-nul

<b>DSS_SINIESTROS_AUTOS: has both keys NUM_SECU_EXPED & NUM_SECU_POL</b>

In [13]:
# checking ID is unique
df_sini["NUM_SECU_EXPED"].nunique() / len(df_sini)

1.0

In [14]:
# checking ID is unique
df_sini["NUM_SECU_POL"].nunique() / len(df_sini)

0.6410001790400831

In [15]:
# checking duplicated rows
df_sini.duplicated().sum()

0

## Loading (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [17]:
# path to dataset
path_df_ret_aseg_cif= "../../../data/interim/5. merged/DM_RETAIL+BTT_ASEGURADOS+tb_cif.feather"

if not(os.path.exists(path_df_ret_aseg_cif) and os.path.isfile(path_df_ret_aseg_cif)):
    raise Exception("File {} doesn't exists.".format(path_df_ret_aseg_cif))

In [18]:
df_ret_aseg_cif = pd.read_feather(path_df_ret_aseg_cif)
df_ret_aseg_cif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741910 entries, 0 to 741909
Data columns (total 39 columns):
ACA                        741910 non-null bool
ANTIG_MESES                741466 non-null float64
APTO_E_MAIL                741910 non-null bool
CANAL                      738563 non-null object
CANAL_ORIGEN               741910 non-null object
CIF_ID                     741909 non-null float64
COBERTURA_PLAN             715940 non-null object
COD_MARCA                  715940 non-null float64
COD_MOD                    715940 non-null float64
DESC_EQUIPO                161921 non-null object
ES_ELEGIBLE                741910 non-null object
FLAG_EBILLING              741910 non-null bool
FLAG_ROBO_CONTENIDO        741910 non-null bool
FOR_COBRO                  741910 non-null object
GRUPO_COMBUSTIBLE          715684 non-null float64
MARCA                      715940 non-null object
MARCA_SIMPLIFICADA         715940 non-null object
MCA_0KM                    732395 non-null

In [19]:
# checking ID is unique
df_ret_aseg_cif["NUM_SECU_POL"].nunique() / len(df_ret_aseg_cif)

1.0

## Comparing DSS_SINIESTROS_AUTOS vs (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)
Checking NUM_SECU_POL intersection

In [1]:
# Checking intersection between both datasets
id_common = set(df_sini["NUM_SECU_POL"].unique()).intersection(set(df_ret_aseg_cif["NUM_SECU_POL"].unique()))
len(id_common), len(id_common) / df_sini["NUM_SECU_POL"].nunique() *100, len(id_common) / df_ret_aseg_cif["NUM_SECU_POL"].nunique()*100

NameError: name 'df_sini' is not defined

In [21]:
not_in_merged = set(df_sini["NUM_SECU_POL"].unique()) - set(df_ret_aseg_cif["NUM_SECU_POL"].unique())
len(not_in_merged)

726777

<h1 style="color:red"> Extremely low intersection</h1>

## Merging DSS_SINIESTROS_AUTOS vs (DM_RETAIL_CLIE+BTT_ASEGURADOS+VIGABT_POLIZAS+tb_cif)

In [24]:
# Merging dropping not founded in DSS_SINIESTROS_AUTOS
df_merged = pd.merge(df_sini, df_ret_aseg_cif, on="NUM_SECU_POL", how="left", suffixes=("_sini", "_ret_aseg_cif"))
print(len(df_sini), len(df_ret_aseg_cif), len(df_merged))

1144995 741910 1144995


In [25]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144995 entries, 0 to 1144994
Data columns (total 82 columns):
CATASTROFICO               1144989 non-null object
CERRADURA_BAUL             504402 non-null object
CERRADURA_DERECHA          504402 non-null object
CERRADURA_IZQUIERDA        504402 non-null object
CODIGO_BAJA                1562 non-null float64
CODIGO_CARATULA            30240 non-null float64
CODIGO_REAPERTURA          58512 non-null float64
CODIGO_VEHICULO            1144905 non-null float64
COD_ACT_BENEF              463893 non-null float64
COD_CAUSA_SINI             1144995 non-null float64
COD_POST_OCURRENCIA        1144991 non-null float64
COD_POST_POLIZA            1144971 non-null float64
COD_POST_TERC              194056 non-null float64
COD_RAMO                   1144995 non-null int64
COD_RIES                   1144995 non-null float64
CONDICION_ROBO_EXP50       13085 non-null float64
DANOS_MATERIALES           1144989 non-null object
DESCRIPCION_TIPO        

### saving merged dataset

In [26]:
path_to_save = "../../../data/interim/5. merged"
check_filename = "DSS_SINIESTROS_AUTOS+DM_RETAIL+BTT_ASEGURADOS+tb_cif.feather"
df_merged.to_feather(os.path.join(path_to_save, check_filename))